In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import os

# Path to your dataset directory
base_dir = '/content/drive/MyDrive/wheat_leaf'  # Update this to your actual path

# Data generators with training/validation split
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,  # Added data augmentation
    width_shift_range=0.2,  # Added data augmentation
    height_shift_range=0.2,  # Added data augmentation
    validation_split=0.2  # Use 20% of the data for validation
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Load pre-trained EfficientNetB0 model + higher level layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer for regularization
x = Dense(1024, activation='relu')(x)  # Increase layer complexity

# Add fully connected layer and a logistic layer with 3 classes
predictions = Dense(3, activation='softmax')(x)

# Model to train
model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze some layers of the base model
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for learning rate scheduling and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stop]
)

# Save the trained model
model.save('wheat_leaf_efficientnet_improved.h5')


Found 327 images belonging to 3 classes.
Found 80 images belonging to 3 classes.
16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/10
11/11 [==============================] - 225s 19s/step - loss: 1.0175 - accuracy: 0.4954 - val_loss: 1.1609 - val_accuracy: 0.2375 - lr: 1.0000e-04
Epoch 2/10
11/11 [==============================] - 109s 10s/step - loss: 0.9693 - accuracy: 0.5566 - val_loss: 1.1475 - val_accuracy: 0.2375 - lr: 1.0000e-04
Epoch 3/10
11/11 [==============================] - 106s 9s/step - loss: 0.8651 - accuracy: 0.6086 - val_loss: 1.1628 - val_accuracy: 0.2375 - lr: 1.0000e-04
Epoch 4/10
11/11 [==============================] - 106s 9s/step - loss: 0.8524 - accuracy: 0.5902 - val_loss: 1.1758 - val_accuracy: 0.2375 - lr: 1.0000e-04
Epoch 5/10
11/11 [==============================] - 108s 10s/step - loss: 0.7505 - accuracy: 0.6758 - val_loss: 1.1646 - val_accuracy: 0.2375 - lr: 1.0000e-04
Epoch 6/10
11/11 [==============================] - 114s 10s/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Load the trained model
model = tf.keras.models.load_model('wheat_leaf_efficientnet.h5')

def predict_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0  # Normalize the image

    # Make predictions
    preds = model.predict(x)
    class_idx = np.argmax(preds[0])

    # Class labels
    class_labels = ['healthy', 'septoria', 'stripe_rust']
    predicted_class = class_labels[class_idx]

    # Print the prediction
    print(f'Image path: {img_path}')
    print('Predicted class:', predicted_class)

    # Display the image
    plt.imshow(img)
    plt.title(f'Predicted class: {predicted_class}')
    plt.axis('off')
    plt.show()

# Example usage
img_path = '/content/drive/MyDrive/wheat_leaf/Healthy/loh(42).JPG'  # Replace with your image path
predict_image(img_path)


OSError: No file or directory found at wheat_leaf_efficientnet.h5